## Suno Demo

Copy-Pasted from: https://colab.research.google.com/drive/1dWWkZzvu7L9Bunq9zvD-W02RFUXoW-Pd?usp=sharing#scrollTo=68QtoUqPWdLk


In [ ]:
#!pip3 install optimum
#!pip install -U flash-attn --no-build-isolation

In [1]:
from transformers import BarkModel, AutoProcessor
import torch

device = "cuda:3"

processor = AutoProcessor.from_pretrained("suno/bark")

#model =  model.to_bettertransformer()
#model = BarkModel.from_pretrained("suno/bark", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)
model = BarkModel.from_pretrained("suno/bark", torch_dtype=torch.float16).to(device)#.to_bettertransformer()

/home/sanyambhutani/.conda/envs/final-checking-meta/lib/python3.12/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [2]:
# prepare the inputs
text_prompt = "Let's try generating speech, with Bark, a text-to-speech model"
inputs = processor(text_prompt)

# generate speech
speech_output = model.generate(**inputs.to(device))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
voice_preset = "v2/en_speaker_6"

# prepare the inputs
text_prompt = "Let's try [laughs] generating speech, with Bark, a text-to-speech model"
inputs = processor(text_prompt, voice_preset=voice_preset)

# generate speech
speech_output = model.generate(**inputs.to(device))

# let's hear it
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [5]:
#for some reason this always gets stuck in a loop
speech_output = model.generate(**inputs, num_beams = 4, temperature = 0.5, semantic_temperature = 0.5)

speech_output = model.generate(**inputs, temperature = 0.5, semantic_temperature = 0.5)

Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.1)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.2)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.3)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.4)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.5)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.6)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.7)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.8)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
speech_output = model.generate(**inputs, temperature = 0.7, semantic_temperature = 0.9)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

## To save

In [ ]:
import scipy

scipy.io.wavfile.write("bark_out.wav", rate=sampling_rate, data=speech_output[0].cpu().numpy())